# Features creation
In this notebook, raw data is loaded and parsed in order to have similar data distributions in train and test sets.

After that, some basic features are computed and the results are saved into files.

In [1]:
import numpy as np
import pandas as pd 
from tqdm import tqdm_notebook
import os
import sys
import datetime
import os.path
from calendar import monthrange

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rc
from cycler import cycler
%matplotlib inline

from dateutil import rrule
from datetime import date
 
mpl.rcParams['axes.prop_cycle'] = cycler('color', ['#ff0000', '#0000ff',   '#00ffff','#ffA300', '#00ff00', 
     '#ff00ff', '#990000', '#009999', '#999900', '#009900', '#009999'])

rc('font', size=16)
rc('font',**{'family':'serif','serif':['Computer Modern']})
rc('text', usetex=False)
rc('figure', figsize=(12, 10))
rc('axes', linewidth=.5)
rc('lines', linewidth=1.75)

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
sys.path.append("../src") # Adds directory to python modules path.
from models import utils
from data import distributions

In [3]:
# Configurations
TARGET_LABEL = 'item_cnt_month'
MONTH_INT_LABEL = 'date_block_num'
INDEX_COLUMNS = ['date_block_num', 'shop_id', 'item_id']
LAGS_WINDOWS = [1, 2, 3, 4, 5, 12]
px_operations = ['last']
operations_dict = {
    'item_cnt_day': ['sum', 'min'],
    # 'date': ['first'],
    'item_price': px_operations
}

## Load data

In [4]:
train = utils.load_training(parse_dts=True)
train.sample(5)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
1444387,2014-02-02,13,37,1855,599.5,1
1256267,2013-12-30,11,47,6084,2199.0,2
1338097,2014-01-07,12,5,1495,599.0,3
2050713,2014-10-25,21,10,17464,200.0,1
37852,2013-01-19,0,0,12134,189.0,1


In [5]:
# Basic data cleaning
train = train[train['item_price'] > 0]
train = train[train['item_cnt_day'] < 2000]

In [6]:
test = utils.load_raw_data('test.csv.gz', index_col='ID')
test.sample(5)

,shop_id,item_id
ID,,
12824,6,7384
115526,49,11535
175173,35,10614
113875,49,22162
164705,36,13009


## Initial Data Parsing
We start by aggregating daily data into monthly data, taking some different measures.

In [7]:
gb_list = ['date_block_num', 'shop_id', 'item_id']
train = train.groupby(gb_list).agg(operations_dict)
# Parse dataframe
train.columns = ['%s_%s' % c for c in train.columns]
train = train.rename(columns={'item_cnt_day_sum': TARGET_LABEL})
train = train.reset_index()
# Show sample of rows
train.sample(10)

,date_block_num,shop_id,item_id,item_cnt_month,item_cnt_day_min,item_price_last
91067,1,27,3432,20,1,599.000000
787831,13,58,18146,1,1,749.000000
205110,3,25,16619,1,1,149.000000
894984,16,22,19294,2,1,1000.000000
236500,3,56,16993,1,1,149.000000
682738,11,57,5865,3,1,1799.000000
1099503,20,51,19759,1,1,69.000000
42191,0,37,19653,2,1,549.000000
980602,18,13,6332,3,1,175.050003
316688,5,27,6916,1,1,749.000000


Now, before computing features, the data distributions should be fixed in order to be similar in train and test sets.

In [8]:
# train, test = distributions.fix_distributions(train, test, TARGET_LABEL)
train = distributions.fix_train_distribution(train, TARGET_LABEL)
train['item_cnt_day_min'] = train['item_cnt_day_min'].fillna(0.0).astype(np.int32)
train['item_cnt_month'] = train['item_cnt_month'].astype(np.int32)
train = utils.downcast_dtypes(train)

In order to compute features for train and test sets at the same time, it will be appended to the train set.

In [9]:
# Append
test['date_block_num'] = train['date_block_num'].max() + 1
data = train.append(test, sort=True)

In [10]:
# Downcast data
data = utils.downcast_dtypes(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128050 entries, 1613016 to 214199
Data columns (total 6 columns):
date_block_num      int32
item_cnt_day_min    float32
item_cnt_month      float32
item_id             int32
item_price_last     float32
shop_id             int32
dtypes: float32(3), int32(3)
memory usage: 339.6 MB


Now we are in conditions to compute features.

The features can be split into these categories:

    1. Lag features: related to previous sales
    2. Prices features: related to previous prices
    3. Date features: related to current month

## Lag features
Let's create columns with lag features, i.e., with value of sales n months before current month.

In [11]:
for l in LAGS_WINDOWS:
    lag_df = data[INDEX_COLUMNS + [TARGET_LABEL]]
    lag_df['date_block_num'] += l
    lag_df = lag_df.rename(columns={TARGET_LABEL: 'lag' + str(l)})
    data = data.merge(lag_df, on=INDEX_COLUMNS, how='left')

## Prices features
Prices features will be, for each pair:
    - minimum price of last month
    - maximum price of last month
    - last price of last month

In [12]:
col_labels = ['item_price_' + op for op in px_operations]
for c in col_labels:
    lag_df = data[INDEX_COLUMNS + [c]]
    lag_df['date_block_num'] += 1
    lag_df = lag_df.rename(columns={c: c + '_lag1'})
    # Remove raw column
    data = data.drop(c, axis=1)
    # Add lagged column
    data = data.merge(lag_df, on=INDEX_COLUMNS, how='left')

## Date features

In [13]:
# Add month
data['month'] = data['date_block_num'].apply(lambda m: 1 + m % 12)
data['month'] = data['month'].astype(np.int16)

In [14]:
m_range = {}
m_weekdays = {}
for yy in [2013, 2014, 2015]:
    m_range[yy] = {}
    m_weekdays[yy] = {}
    for mm in range(1, 12 + 1):
        rng = monthrange(yy, mm)[1]
        m_range[yy][mm] = rng
        # Compute number of week days in month
        start = date(yy, mm, 1)
        end = date(yy, mm, rng)
        weekdays = rrule.rrule(
            rrule.DAILY, byweekday=range(0, 5), dtstart=start, until=end
        )
        m_weekdays[yy][mm] = len(list(weekdays))

m_ranges = [m_range[yy][mm] for yy in [2013, 2014, 2015] for mm in range(1, 12 + 1)]
m_weekend = [
    m_range[yy][mm] - m_weekdays[yy][mm] 
    for yy in [2013, 2014, 2015] for mm in range(1, 12 + 1)
]
data['month_range'] = data[MONTH_INT_LABEL].map(pd.Series(m_ranges))
data['n_weekend'] = data[MONTH_INT_LABEL].map(pd.Series(m_weekend))

## Item categories
Now, item categories features will be added to the features DataFrame

In [15]:
# Load files
items = utils.load_raw_data('items.csv')
item_cats = utils.load_raw_data('item_categories.csv')

In [16]:
from itertools import zip_longest
# Parse category names
s = item_cats['item_category_name'].str.split('-')
cat_names = pd.DataFrame.from_records(zip_longest(*s.values)).T
cat_names.columns = ['catname1', 'catname2', 'catname3']
# To save memory, let's encode category names
for c in cat_names.columns:
    vals = cat_names[c].unique()
    name_enc = dict(zip(vals, range(len(vals))))
    cat_names[c] = cat_names[c].apply(lambda x: name_enc[x])
# Join DataFrames
item_cats_exp = item_cats.join(cat_names)

In [17]:
# Merge
items_df = items.merge(item_cats_exp)
# Filter columns
cols = ['item_id', 'item_category_id', 'catname1', 'catname2']
# cols = ['item_id', 'item_category_id']
items_df = items_df[cols]
items_df = utils.downcast_dtypes(items_df)

In [18]:
data = data.merge(items_df, on='item_id')

# Save features
Finally, split into train and test and save to hdf5 files.

In [19]:
# Downcast dtypes
for c in ['date_block_num', 'shop_id', 'item_category_id', 'month', 'month_range', 'n_weekend']:
    data[c] = data[c].astype(np.int8)
for c in ['item_id']:
    data[c] = data[c].astype(np.int16)
for c in ['item_cnt_day_min', 'lag1', 'lag2', 'lag3', 'lag12', 'item_price_last_lag1']:
    data[c] = data[c].astype(np.float16)
for c in ['item_cnt_month']:
    data[c] = data[c].astype(np.float32)

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128050 entries, 0 to 11128049
Data columns (total 18 columns):
date_block_num          int8
item_cnt_day_min        float16
item_cnt_month          float32
item_id                 int16
shop_id                 int8
lag1                    float16
lag2                    float16
lag3                    float16
lag4                    float32
lag5                    float32
lag12                   float16
item_price_last_lag1    float16
month                   int8
month_range             int8
n_weekend               int8
item_category_id        int8
catname1                int32
catname2                int32
dtypes: float16(6), float32(3), int16(1), int32(2), int8(6)
memory usage: 509.4 MB


In [21]:
last_m = data['date_block_num'].max()
train_feats = data[data['date_block_num'] < last_m]
test_feats = data[data['date_block_num'] == last_m].drop(TARGET_LABEL, axis=1)

In [22]:
# Save to an HDF5 file
data_path = '../data/processed/'
## Save train_feats in chunks because of memory
#def chunker(seq, size):
#    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

#hdf_key = 'hdf_key'
# df_cols_to_index = [...]  # list of columns (labels) that should be indexed
#df_cols_to_index = list(train_feats.columns)
#store = pd.HDFStore(data_path + 'train_features.h5')
#for i, tf in enumerate(chunker(train_feats, int(len(train_feats) / 3))):
#    print('chunk number %d' % i)
#    # don't index data columns in each iteration - we'll do it later ...
#    store.append(hdf_key, tf, data_columns=df_cols_to_index, index=False)
#    # index data columns in HDFStore
#
#print('creating table index')
#store.create_table_index(hdf_key, columns=df_cols_to_index, optlevel=9, kind='full')
#print('closing')
#store.close()
# Save test features
train_feats.to_hdf(data_path + 'train_features.h5', key='df', mode='w')
test_feats.to_hdf(data_path + 'test_features.h5', key='df', mode='w')